In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from src.models.HandcraftedModel import HandcraftedModel
from src.DataManager import DataManager

from src.models.Model import IMAGE_INPUT_SIZE
from src.config import IMDB_CROPPED_PATH, IMBD_CROPPED_METADATA_FILENAME

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [7]:
N_SAMPLE = .001

# Read the data
data_manager = DataManager(IMDB_CROPPED_PATH, IMBD_CROPPED_METADATA_FILENAME, IMAGE_INPUT_SIZE,
                           n_subset=N_SAMPLE, normalize_images=False, normalize_age=True)
data = data_manager.get_dataset()

# Split into train, validation, test
train, validation, test = data_manager.split_dataset(data)

# Read images
print('Read training images ...')
X_train, y_train = data_manager.get_X(train), data_manager.get_y(train)
print('Read validation images ...')
X_val, y_val = data_manager.get_X(validation), data_manager.get_y(validation)
print('Read test images ...')
X_test, y_test = data_manager.get_X(test), data_manager.get_y(test)

n_sift = 150
color_hist_bins = 128
lbp_n_points = 24
lbp_radius = 3
handcrafted_model = HandcraftedModel(n_sift, color_hist_bins, lbp_n_points, lbp_radius)

# EXTRACTIONS
df_train = handcrafted_model.extract_dataset_features(X_train, y_train)
df_val = handcrafted_model.extract_dataset_features(X_val, y_val)
df_test = handcrafted_model.extract_dataset_features(X_test, y_test)

# CLEANING FROM NANS
data_manager.delete_nan_columns(df_train, df_val, df_test)

Read training images ...


100%|██████████| 16709/16709 [01:55<00:00, 144.18it/s]


Read validation images ...


100%|██████████| 2949/2949 [00:19<00:00, 152.02it/s]


Read test images ...


100%|██████████| 8426/8426 [02:09<00:00, 65.16it/s] 


Extracting dataset features ...


 41%|████      | 6777/16709 [10:31<15:26, 10.73it/s]  
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\lucre\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lucre\AppData\Local\Temp/ipykernel_5788/2810727123.py", line 26, in <module>
    df_train = handcrafted_model.extract_dataset_features(X_train, y_train)
  File "C:\Users\lucre\PycharmProjects\visual-processing-project\src\models\HandcraftedModel.py", line 62, in extract_dataset_features
    x = cv2.bilateralFilter(x, 15, 75, 75)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\lucre\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, 

TypeError: object of type 'NoneType' has no len()

In [5]:
df_train.shape

(0, 0)

In [ ]:
# srotolo
df_train_new = pd.DataFrame()
for _, row in df_train.iterrows():
    d = {}
    i = 0
    for col in range(26):
        d[i] = row[col]
        i = i + 1
    for col in range(26, len(df_train.columns) - 2):
        for el in row[col]:
            d[i] = el
            i = i + 1
    df_train_new = df_train_new.append(d, ignore_index=True)

df_val_new = pd.DataFrame()
for _, row in df_val.iterrows():
    d = {}
    i = 0
    for col in range(26):
        d[i] = row[col]
        i = i + 1
    for col in range(26, len(df_val.columns) - 2):
        for el in row[col]:
            d[i] = el
            i = i + 1
    df_val_new = df_val_new.append(d, ignore_index=True)

df_test_new = pd.DataFrame()
for _, row in df_test.iterrows():
    d = {}
    i = 0
    for col in range(26):
        d[i] = row[col]
        i = i + 1
    for col in range(26, len(df_test.columns) - 2):
        for el in row[col]:
            d[i] = el
            i = i + 1
    df_test_new = df_test_new.append(d, ignore_index=True)

In [ ]:
print(df_train_new.shape)
print(df_val_new.shape)
print(df_test_new.shape)

In [ ]:
clf = SVC()
clf.fit(df_train_new, df_train["gender"])

In [ ]:
preds = clf.predict(df_test_new)
acc_test = accuracy_score(df_test["gender"], preds)
conf_mat_test = confusion_matrix(df_test["gender"], preds)
print(acc_test)
print(conf_mat_test)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=3,algorithm='brute')
classifier.fit(df_train_new, df_train["gender"])
preds = classifier.predict(df_test_new)
acc_test = accuracy_score(df_test["gender"], preds)
conf_mat_test = confusion_matrix(df_test["gender"], preds)
print(acc_test)
print(conf_mat_test)